In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


df_cleaned = df.dropna(subset=['price']).copy()


columns_to_drop = ['id', 'product_id']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)


df_processed = pd.get_dummies(
    df_cleaned, 
    columns=['cab_type', 'destination', 'source', 'name'], 
    drop_first=True
)

df_processed = df_processed.dropna(axis=1, how='all')

In [41]:
X = df_processed.drop('price', axis=1) 
y = df_processed['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15)
model.fit(X_train, y_train) 

print("\nModel trained")


Model trained


In [42]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared Score (R2): {r2:.4f}')


errors = np.abs(y_test - y_pred)


error_df = pd.DataFrame({
    'Actual_Price': y_test, 
    'Predicted_Price': y_pred, 
    'Absolute_Error': errors
})


top_5_indices = error_df['Absolute_Error'].nlargest(5).index


top_5_errors_X = X_test.loc[top_5_indices]


top_5_samples = pd.concat([top_5_errors_X, error_df.loc[top_5_indices]], axis=1)

Mean Absolute Error (MAE): 1.13
R-squared Score (R2): 0.9657


In [43]:
df_top5 = pd.read_csv('top_5_error_samples.csv', index_col=0)

name_cols = [col for col in df_top5.columns if col.startswith('name_')]
df_top5['Service_Type'] = df_top5.apply(
    lambda row: [col.replace('name_', '') for col in name_cols if row[col]][0], 
    axis=1
)

table_data = df_top5[[
    'Actual_Price', 
    'Predicted_Price', 
    'Absolute_Error', 
    'distance', 
    'surge_multiplier', 
    'Service_Type'
]].round(2)

table_data = table_data.sort_values(by='Absolute_Error', ascending=False)
print(table_data.to_markdown(index=True))

|        |   Actual_Price |   Predicted_Price |   Absolute_Error |   distance |   surge_multiplier | Service_Type   |
|-------:|---------------:|------------------:|-----------------:|-----------:|-------------------:|:---------------|
| 164401 |           89.5 |             48.63 |            40.87 |       7.46 |                  1 | Black SUV      |
| 658443 |           58   |             18.85 |            39.15 |       3.07 |                  1 | UberXL         |
|   2686 |           56   |             19.2  |            36.8  |       3.23 |                  1 | UberXL         |
| 510312 |           65   |             30.92 |            34.08 |       2.32 |                  1 | Black SUV      |
| 119248 |           60.5 |             26.94 |            33.56 |       1.35 |                  1 | Black SUV      |
